# 数据读取部分

In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.utils import shuffle
from tqdm import tqdm
import math
import sys
import os

In [2]:
print("本地路径 = ", sys.argv[0], "\n")

path1 = os.getcwd() #获取当前工作目录路径
path2 = os.path.abspath('.') #获取当前工作目录路径
path3 = os.path.abspath('test.txt') #获取当前目录文件下的工作目录路径
path4 = os.path.abspath('..') #获取当前工作的父目录 ！注意是父目录路径
path5 = os.path.abspath(os.curdir) #获取当前工作目录路径

print("当前的代码绝对路径 = ",path1)
print("获取当前工作目录路径 = ",path2)
print("获取当前目录文件下的工作目录路径 = ",path3)
print("获取当前工作的父目录 = ",path4)
print("获取当前工作目录路径 = ",path5)

def show_file(file_dir):
    
    print("\n\nFiles in your current working directory:")
    dirlist = os.listdir(os.getcwd())
    for i, file in enumerate(dirlist):
        print(f"  [{i}] ", file)
        
show_file(os.getcwd())

本地路径 =  d:\python\lib\site-packages\ipykernel_launcher.py 

当前的代码绝对路径 =  C:\Users\程琪皓\Desktop\相关系数思路
获取当前工作目录路径 =  C:\Users\程琪皓\Desktop\相关系数思路
获取当前目录文件下的工作目录路径 =  C:\Users\程琪皓\Desktop\相关系数思路\test.txt
获取当前工作的父目录 =  C:\Users\程琪皓\Desktop
获取当前工作目录路径 =  C:\Users\程琪皓\Desktop\相关系数思路


Files in your current working directory:
  [0]  .ipynb_checkpoints
  [1]  data_by_artist.csv
  [2]  data_by_artist.xlsx
  [3]  data_processing.ipynb
  [4]  gener_classify.ipynb
  [5]  influence_data.csv
  [6]  influence_data.xlsx
  [7]  ~$data_by_artist.xlsx


# Read in the file

In [3]:
influence_data = pd.read_csv("influence_data.csv")
influence_data.head()

,influencer_id,influencer_name,influencer_main_genre,influencer_active_start,follower_id,follower_name,follower_main_genre,follower_active_start
0,1163,Serge Chaloff,Jazz,1930,542549,Gerry Mulligan,Jazz,1940
1,2130,Alfred Newman,Stage & Screen,1930,678009,Henry Mancini,Easy Listening,1940
2,2130,Alfred Newman,Stage & Screen,1930,394806,Randy Newman,Pop/Rock,1960
3,2130,Alfred Newman,Stage & Screen,1930,330634,Jerry Goldsmith,Stage & Screen,1950
4,2130,Alfred Newman,Stage & Screen,1930,327765,John Barry,Stage & Screen,1950


In [4]:
dataFrame_artist = pd.read_csv("data_by_artist.csv")
dataFrame_artist.head()

,artist_name,artist_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,duration_ms,popularity,count
0,Frank Sinatra,792507,0.384478,0.238017,0.364288,110.181698,-14.271141,1,5,0.735648,0.020855,0.232106,0.049614,189179.9255,26.004383,1369
1,Vladimir Horowitz,119107,0.343210,0.118844,0.225951,94.900679,-23.193418,1,1,0.990070,0.879508,0.183812,0.043360,266541.1251,3.592378,1207
2,Johnny Cash,816890,0.619803,0.449381,0.680662,115.037747,-11.593104,1,10,0.685637,0.022647,0.242243,0.098216,162279.2672,26.614130,1104
3,Billie Holiday,79016,0.572637,0.201368,0.498934,109.912172,-13.225966,1,5,0.908499,0.013064,0.217727,0.062432,185131.4530,15.621005,1095
4,Bob Dylan,66915,0.512598,0.477932,0.551934,126.160149,-11.184330,1,7,0.562567,0.034211,0.308978,0.064535,256713.4203,30.860806,1092


# 首先从influence data当中获得一个 artists 到genra 的映射

In [5]:
influence_col = influence_data.loc[:,["influencer_id","influencer_name","influencer_main_genre","influencer_active_start"]]
influence_col.columns =["id","name","genre","year"]
influence_col.head()
follower_col = influence_data.loc[:,["follower_id","follower_name","follower_main_genre","follower_active_start"]]
follower_col.columns =["id","name","genre","year"]
person_col = pd.concat([influence_col,follower_col])

# per_col.index=range(85540)
person_col.shape
person_col.head()

,id,name,genre,year
0,1163,Serge Chaloff,Jazz,1930
1,2130,Alfred Newman,Stage & Screen,1930
2,2130,Alfred Newman,Stage & Screen,1930
3,2130,Alfred Newman,Stage & Screen,1930
4,2130,Alfred Newman,Stage & Screen,1930


In [6]:
per_map=person_col.drop_duplicates(["id"])
per_map.head()

,id,name,genre,year
0,1163,Serge Chaloff,Jazz,1930
1,2130,Alfred Newman,Stage & Screen,1930
7,3413,Allan Jones,Vocal,1930
8,3829,T-Bone Walker,Blues,1930
56,11028,Frank Loesser,Stage & Screen,1930


In [7]:
def getGenre(id):
    try:
        return per_map[per_map.id==id].iloc[0,2]
    except:
        return "Unknown"
getGenre(12138)

'Unknown'

# 其次将artist表当中的artist添加流派一列，并且按照流派进行区分

# 在artist后面添加一列genre

## 建立一个字典genre_map,将数字对应到不同的流派

In [8]:
genre_map=influence_data.drop_duplicates(["follower_main_genre"])["follower_main_genre"]
genre_map.index=range(20)
genre_map=genre_map.to_dict()
genre_map # genre对应的字典

{0: 'Jazz',
 1: 'Easy Listening',
 2: 'Pop/Rock',
 3: 'Stage & Screen',
 4: 'Vocal',
 5: 'R&B;',
 6: 'Blues',
 7: 'Folk',
 8: 'Religious',
 9: 'International',
 10: 'Country',
 11: 'Latin',
 12: 'Classical',
 13: 'Reggae',
 14: 'Comedy/Spoken',
 15: 'Avant-Garde',
 16: 'Electronic',
 17: "Children's",
 18: 'New Age',
 19: 'Unknown'}

# 建立一个字典re_genre_map，将流派对应回数字

In [9]:
re_genre_map={}
for i,key in enumerate(genre_map):
    re_genre_map[genre_map[key]]=key
re_genre_map

{'Jazz': 0,
 'Easy Listening': 1,
 'Pop/Rock': 2,
 'Stage & Screen': 3,
 'Vocal': 4,
 'R&B;': 5,
 'Blues': 6,
 'Folk': 7,
 'Religious': 8,
 'International': 9,
 'Country': 10,
 'Latin': 11,
 'Classical': 12,
 'Reggae': 13,
 'Comedy/Spoken': 14,
 'Avant-Garde': 15,
 'Electronic': 16,
 "Children's": 17,
 'New Age': 18,
 'Unknown': 19}

In [10]:
dataFrame_artist.head()

,artist_name,artist_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,duration_ms,popularity,count
0,Frank Sinatra,792507,0.384478,0.238017,0.364288,110.181698,-14.271141,1,5,0.735648,0.020855,0.232106,0.049614,189179.9255,26.004383,1369
1,Vladimir Horowitz,119107,0.343210,0.118844,0.225951,94.900679,-23.193418,1,1,0.990070,0.879508,0.183812,0.043360,266541.1251,3.592378,1207
2,Johnny Cash,816890,0.619803,0.449381,0.680662,115.037747,-11.593104,1,10,0.685637,0.022647,0.242243,0.098216,162279.2672,26.614130,1104
3,Billie Holiday,79016,0.572637,0.201368,0.498934,109.912172,-13.225966,1,5,0.908499,0.013064,0.217727,0.062432,185131.4530,15.621005,1095
4,Bob Dylan,66915,0.512598,0.477932,0.551934,126.160149,-11.184330,1,7,0.562567,0.034211,0.308978,0.064535,256713.4203,30.860806,1092


In [11]:
tmp=0
for i,r in dataFrame_artist.iterrows():
    Id=r["artist_id"]
    dataFrame_artist.loc[i,"genre"]=getGenre(Id)
    if dataFrame_artist.loc[i,"genre"]!=-1:
        dataFrame_artist.loc[i,"genre_id"]=re_genre_map[dataFrame_artist.loc[i,"genre"]]
    else:
        dataFrame_artist.loc[i,"genre_id"]=-1
dataFrame_artist

,artist_name,artist_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,duration_ms,popularity,count,genre,genre_id
0,Frank Sinatra,792507,0.384478,0.238017,0.364288,110.181698,-14.271141,1,5,0.735648,0.020855,0.232106,0.049614,189179.9255,26.004383,1369,Vocal,4.0
1,Vladimir Horowitz,119107,0.343210,0.118844,0.225951,94.900679,-23.193418,1,1,0.990070,0.879508,0.183812,0.043360,266541.1251,3.592378,1207,Unknown,19.0
2,Johnny Cash,816890,0.619803,0.449381,0.680662,115.037747,-11.593104,1,10,0.685637,0.022647,0.242243,0.098216,162279.2672,26.614130,1104,Country,10.0
3,Billie Holiday,79016,0.572637,0.201368,0.498934,109.912172,-13.225966,1,5,0.908499,0.013064,0.217727,0.062432,185131.4530,15.621005,1095,Vocal,4.0
4,Bob Dylan,66915,0.512598,0.477932,0.551934,126.160149,-11.184330,1,7,0.562567,0.034211,0.308978,0.064535,256713.4203,30.860806,1092,Pop/Rock,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5849,Natalie La Rose,3359519,0.830000,0.520000,0.735000,104.990000,-8.714000,1,0,0.000792,0.000013,0.065600,0.037600,189907.0000,64.000000,1,R&B;,5.0
5850,Sarah Ross,3381566,0.721000,0.944000,0.626000,85.002000,-5.982000,1,8,0.013000,0.000000,0.320000,0.159000,262760.0000,52.000000,1,Country,10.0
5851,Rotimi,3410250,0.637000,0.501000,0.431000,103.993000,-6.148000,0,0,0.229000,0.000059,0.099000,0.187000,185461.0000,71.000000,1,R&B;,5.0
5852,Jillian Jacqueline,3455945,0.547000,0.672000,0.283000,155.791000,-5.023000,1,11,0.304000,0.000000,0.099600,0.049600,213133.0000,58.000000,1,Country,10.0


In [12]:
dataFrame_artist = dataFrame_artist.sort_values(by=['genre_id', 'artist_id'], ascending=True)
dataFrame_artist

,artist_name,artist_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,duration_ms,popularity,count,genre,genre_id
4617,Serge Chaloff,1163,0.348000,0.291000,0.586500,193.041000,-15.804500,1,1,0.969000,6.615000e-01,0.252000,0.089850,627330.0000,0.000000,2,Jazz,0.0
1127,Alice Coltrane,6143,0.330837,0.376828,0.322158,97.043512,-15.849837,1,9,0.655507,3.261672e-01,0.158465,0.042798,414027.0698,28.674419,43,Jazz,0.0
2623,Stan Levey,6563,0.443333,0.429233,0.483833,130.285500,-12.525833,1,5,0.716500,1.356357e-01,0.159317,0.040117,242520.8333,1.500000,12,Jazz,0.0
55,Charles Mingus,9680,0.440410,0.302933,0.417417,110.290904,-15.562311,1,5,0.691750,2.385482e-01,0.220114,0.063169,393268.1066,16.639344,366,Jazz,0.0
3964,Stanley Jordan,11910,0.457000,0.383000,0.485000,125.591000,-17.352500,0,10,0.666000,8.085000e-01,0.118200,0.051450,404893.5000,31.500000,4,Jazz,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5663,2NE1,2693354,0.819000,0.844000,0.687000,127.997000,-3.739000,1,0,0.007670,8.340000e-03,0.056000,0.038000,208731.0000,64.000000,2,Unknown,19.0
946,Glass Animals,2984484,0.618907,0.502519,0.373696,126.844463,-9.469463,0,1,0.296569,4.260265e-02,0.164104,0.051589,251207.4815,57.851852,54,Unknown,19.0
1328,Bebe Rexha,2992951,0.626861,0.676139,0.433028,114.470556,-5.464083,0,11,0.134307,3.630000e-07,0.175958,0.081108,197545.5833,69.611111,36,Unknown,19.0
3145,Nico & Vinz,3228134,0.649889,0.711444,0.679000,115.101444,-5.410667,1,8,0.208933,8.240000e-07,0.249111,0.046389,229515.8889,63.333333,9,Unknown,19.0


In [13]:
genre_number = dataFrame_artist["genre_id"].value_counts()
value = genre_number.values
index = genre_number.index.values
genre_dict = {}
for i in range(len(value)):
    genre_dict[index[i]] = value[i]
    
genre_num = []
for i in range(len(value)):
    genre_num.append([i,genre_dict[i]])
genre_num = np.array(genre_num).T[-1]
genre_position = np.cumsum(genre_num)
print("genre_num = \n", genre_num)
print("genre_position = \n", genre_position)

genre_num = 
 [ 406   23 2807   50  162  677  101   95   89   81  403  229   28  141
   46   11  208    4   38  255]
genre_position = 
 [ 406  429 3236 3286 3448 4125 4226 4321 4410 4491 4894 5123 5151 5292
 5338 5349 5557 5561 5599 5854]


## Kernel Code
下面是代码的主要部分，用于计算相关系数矩阵，并且对相关系数进行统计，注意到计算的时候只需要用到其中的某一些列

In [69]:
dataFrame_artist

,artist_name,artist_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,duration_ms,popularity,count,genre,genre_id
4617,Serge Chaloff,1163,0.348000,0.291000,0.586500,193.041000,-15.804500,1,1,0.969000,6.615000e-01,0.252000,0.089850,627330.0000,0.000000,2,Jazz,0.0
1127,Alice Coltrane,6143,0.330837,0.376828,0.322158,97.043512,-15.849837,1,9,0.655507,3.261672e-01,0.158465,0.042798,414027.0698,28.674419,43,Jazz,0.0
2623,Stan Levey,6563,0.443333,0.429233,0.483833,130.285500,-12.525833,1,5,0.716500,1.356357e-01,0.159317,0.040117,242520.8333,1.500000,12,Jazz,0.0
55,Charles Mingus,9680,0.440410,0.302933,0.417417,110.290904,-15.562311,1,5,0.691750,2.385482e-01,0.220114,0.063169,393268.1066,16.639344,366,Jazz,0.0
3964,Stanley Jordan,11910,0.457000,0.383000,0.485000,125.591000,-17.352500,0,10,0.666000,8.085000e-01,0.118200,0.051450,404893.5000,31.500000,4,Jazz,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5663,2NE1,2693354,0.819000,0.844000,0.687000,127.997000,-3.739000,1,0,0.007670,8.340000e-03,0.056000,0.038000,208731.0000,64.000000,2,Unknown,19.0
946,Glass Animals,2984484,0.618907,0.502519,0.373696,126.844463,-9.469463,0,1,0.296569,4.260265e-02,0.164104,0.051589,251207.4815,57.851852,54,Unknown,19.0
1328,Bebe Rexha,2992951,0.626861,0.676139,0.433028,114.470556,-5.464083,0,11,0.134307,3.630000e-07,0.175958,0.081108,197545.5833,69.611111,36,Unknown,19.0
3145,Nico & Vinz,3228134,0.649889,0.711444,0.679000,115.101444,-5.410667,1,8,0.208933,8.240000e-07,0.249111,0.046389,229515.8889,63.333333,9,Unknown,19.0


In [104]:
dataFrame_artist_character_vector = dataFrame_artist.loc[:,["danceability","energy","valence","tempo", "loudness","mode","key","genre_id"]].sort_values(by=['genre_id'], ascending=True)

Key_places = {0:0, 7:1, 2:2, 9:3, 4:4, 11:5,
              6:6, 1:7, 8:8, 3:9,10:10, 5:11}

def dot_product_7D(vec1, vec2):
    dot1 = np.sum(vec1[:5] * vec2[:5])
    dot2 = 1 if vec1[5] == vec2[5] else -1
    dot3 = math.cos(np.abs(Key_places[vec1[-1]] - Key_places[vec2[-1]]) * math.pi/6)
    return dot1 + dot2 + dot3

def dot_product_7D_Version2(vec1, vec2):
    dot1 = np.sum(vec1[:5] * vec2[:5])
    dot2 = 1 if vec1[5] == vec2[5] else -1
    dot3 = math.cos(np.abs(Key_places[vec1[-1]] - Key_places[vec2[-1]]) * math.pi/6)
    return dot1 + (dot2 + dot3)/2

def length_7D(vec):
    return np.sqrt(np.sum(vec[:5]**2) + 2)

def length_7D_Version2(vec):
    return np.sqrt(np.sum(vec[:5]**2) + 1)

def cos_7D(vec1, vec2):
    return dot_product_7D(vec1, vec2) / (length_7D(vec1) * length_7D(vec2))

def cos_7D_Version2(vec1, vec2):
    return dot_product_7D_Version2(vec1, vec2) / (length_7D_Version2(vec1) * length_7D_Version2(vec2))
    
def test():    
    vec = np.array([1,1,1,1,1,1,6])
    vec2 = np.array([1,1,1,1,1,1,6])
    print(length_7D(vec) ** 2)
    print(dot_product_7D(vec,vec2))
    print(cos_7D(vec, vec2))

# test()
dataFrame_artist_character_vector

,danceability,energy,valence,tempo,loudness,mode,key,genre_id
4617,0.348000,0.291000,0.586500,193.041000,-15.804500,1,1,0.0
2757,0.571750,0.401333,0.389333,112.109500,-15.766667,1,6,0.0
1210,0.589125,0.204590,0.598300,114.501475,-13.173450,1,5,0.0
5292,0.484000,0.029100,0.379000,98.434000,-23.363000,1,10,0.0
5288,0.521000,0.112000,0.123000,113.168000,-15.672000,1,9,0.0
...,...,...,...,...,...,...,...,...
5001,0.516000,0.962000,0.620000,131.818000,-2.528000,0,2,19.0
5761,0.446000,0.164000,0.138000,87.063000,-18.817000,1,0,19.0
5004,0.653000,0.904000,0.391000,135.073000,-7.637000,0,9,19.0
4244,0.460000,0.892500,0.597000,134.014000,-6.964000,1,2,19.0


# 正则化

In [96]:
vector_samples = np.array(dataFrame_artist_character_vector.values.tolist())[:,:-1]
vector_shape = vector_samples.shape
artist_num = vector_shape[0]

for i in range(5):
    sample = vector_samples[:,i]
    sample = (sample - sample.mean()) / np.sqrt(sample.var())
    vector_samples[:,i] = sample
    print(sample.shape)
    


print("vector_shape = ", vector_shape)
print("artist_num = " , artist_num)

(5854,)
(5854,)
(5854,)
(5854,)
(5854,)
vector_shape =  (5854, 7)
artist_num =  5854


# 计算邻接矩阵

In [97]:
relative_matrix = np.zeros([artist_num, artist_num])

for i in tqdm(range(artist_num),):
    vec_1 = vector_samples[i]
    for j in range(artist_num):
        vec_2 = vector_samples[j]
        relative_matrix[i][j] = cos_7D(vec_1, vec_2)
print(relative_matrix[5][5])

100%|██████████████████████████████████████████████████████████████████████████████| 5854/5854 [12:16<00:00,  7.94it/s]

1.0


In [100]:
dataFrame_relative_matrix = pd.DataFrame(relative_matrix)
dataFrame_relative_matrix.to_csv("matrix/relative_matrix.csv")

In [101]:
print(genre_position)
head = 0
range_list = []
genre_num = len(genre_position)
for i in range(genre_num):
    range_list.append([head, genre_position[i]])
    head = genre_position[i]
print(range_list)

[ 406  429 3236 3286 3448 4125 4226 4321 4410 4491 4894 5123 5151 5292
 5338 5349 5557 5561 5599 5854]
[[0, 406], [406, 429], [429, 3236], [3236, 3286], [3286, 3448], [3448, 4125], [4125, 4226], [4226, 4321], [4321, 4410], [4410, 4491], [4491, 4894], [4894, 5123], [5123, 5151], [5151, 5292], [5292, 5338], [5338, 5349], [5349, 5557], [5557, 5561], [5561, 5599], [5599, 5854]]


In [102]:
mean_matrix = np.zeros([genre_num,genre_num])
for genre_idx1, range1 in enumerate(range_list):
    for genre_idx2, range2 in enumerate(range_list):
        mean_matrix[genre_idx1][genre_idx2] = np.mean(relative_matrix[range1[0]:range1[1], range2[0]:range2[1]])
dataFrame_mean_matrix = pd.DataFrame(mean_matrix)
dataFrame_mean_matrix.to_csv("matrix/mean_matrix.csv")
dataFrame_mean_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.310476,0.351344,-0.046500,0.336633,0.355647,0.029959,0.186252,0.336424,0.061282,0.166365,0.115018,0.029379,0.398954,0.016102,0.130562,0.331237,-0.085236,0.276665,0.337832,0.072693
1,0.351344,0.486032,0.028437,0.499832,0.477100,-0.007996,0.192305,0.421994,0.181493,0.175827,0.158409,-0.002568,0.557398,-0.103812,0.181319,0.421056,-0.096038,0.276120,0.464977,0.104547
2,-0.046500,0.028437,0.182093,-0.020221,-0.003885,0.039262,0.067790,0.022013,0.123901,0.033776,0.152448,0.087553,-0.035503,0.009135,0.059945,-0.015635,0.069161,0.039913,-0.040838,0.075682
3,0.336633,0.499832,-0.020221,0.628054,0.500632,-0.114301,0.080297,0.404409,0.178070,0.112828,0.030784,-0.129574,0.680078,-0.281085,0.176460,0.441890,-0.147268,0.119312,0.579252,0.054581
4,0.355647,0.477100,-0.003885,0.500632,0.494558,0.003946,0.199827,0.440104,0.173736,0.185271,0.153258,-0.000834,0.552876,-0.085263,0.162216,0.409464,-0.116065,0.305000,0.462861,0.093964
5,0.029959,-0.007996,0.039262,-0.114301,0.003946,0.152386,0.133598,0.048606,0.026413,0.096329,0.151979,0.155310,-0.111763,0.233330,0.045871,-0.017628,0.076185,0.227992,-0.100091,0.063542
6,0.186252,0.192305,0.067790,0.080297,0.199827,0.133598,0.277217,0.263712,0.051614,0.198887,0.256022,0.189052,0.117938,0.192608,0.090189,0.163630,-0.016992,0.343691,0.094577,0.114502
7,0.336424,0.421994,0.022013,0.404409,0.440104,0.048606,0.263712,0.468061,0.132642,0.221025,0.222337,0.061850,0.475747,0.011290,0.160964,0.390043,-0.082387,0.361932,0.414059,0.114220
8,0.061282,0.181493,0.123901,0.178070,0.173736,0.026413,0.051614,0.132642,0.212756,0.055497,0.123140,0.016894,0.167912,-0.060510,0.119836,0.110808,0.035454,0.112127,0.123314,0.073282
9,0.166365,0.175827,0.033776,0.112828,0.185271,0.096329,0.198887,0.221025,0.055497,0.158625,0.175831,0.121964,0.146817,0.128620,0.106757,0.157476,-0.014829,0.268601,0.119720,0.085763


# 另外一种计算邻接矩阵的方法

In [105]:
relative_matrix = np.zeros([artist_num, artist_num])

for i in tqdm(range(artist_num),):
    vec_1 = vector_samples[i]
    for j in range(artist_num):
        vec_2 = vector_samples[j]
        relative_matrix[i][j] = cos_7D_Version2(vec_1, vec_2)
print(relative_matrix[5][5])

100%|██████████████████████████████████████████████████████████████████████████████| 5854/5854 [12:09<00:00,  8.03it/s]

1.0000000000000002


In [106]:
dataFrame_relative_matrix_Version2 = pd.DataFrame(relative_matrix)
dataFrame_relative_matrix_Version2.to_csv("matrix/relative_matrix_Version2.csv")

In [107]:
print(genre_position)
head = 0
range_list = []
genre_num = len(genre_position)
for i in range(genre_num):
    range_list.append([head, genre_position[i]])
    head = genre_position[i]
print(range_list)

[ 406  429 3236 3286 3448 4125 4226 4321 4410 4491 4894 5123 5151 5292
 5338 5349 5557 5561 5599 5854]
[[0, 406], [406, 429], [429, 3236], [3236, 3286], [3286, 3448], [3448, 4125], [4125, 4226], [4226, 4321], [4321, 4410], [4410, 4491], [4491, 4894], [4894, 5123], [5123, 5151], [5151, 5292], [5292, 5338], [5338, 5349], [5349, 5557], [5557, 5561], [5561, 5599], [5599, 5854]]


In [108]:
mean_matrix = np.zeros([genre_num,genre_num])
for genre_idx1, range1 in enumerate(range_list):
    for genre_idx2, range2 in enumerate(range_list):
        mean_matrix[genre_idx1][genre_idx2] = np.mean(relative_matrix[range1[0]:range1[1], range2[0]:range2[1]])
dataFrame_mean_matrix_Version2 = pd.DataFrame(mean_matrix)
dataFrame_mean_matrix_Version2.to_csv("matrix/dataFrame_mean_matrix_Version2.csv")
dataFrame_mean_matrix_Version2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.333224,0.363235,-0.085625,0.356504,0.375633,0.020839,0.184529,0.361444,0.040363,0.171097,0.090681,0.008551,0.417476,0.005287,0.136139,0.338473,-0.109434,0.288966,0.361353,0.059182
1,0.363235,0.481619,-0.032068,0.520862,0.486405,-0.041567,0.159213,0.428541,0.146414,0.157648,0.096270,-0.052703,0.573934,-0.144266,0.169023,0.417452,-0.132623,0.247913,0.489398,0.072453
2,-0.085625,-0.032068,0.147715,-0.058863,-0.060306,0.011069,0.011833,-0.046132,0.085302,-0.009751,0.087202,0.050749,-0.079089,-0.017557,0.020943,-0.058022,0.059342,-0.039161,-0.077490,0.041845
3,0.356504,0.520862,-0.058863,0.664081,0.531574,-0.149821,0.052416,0.422348,0.173151,0.097404,-0.015785,-0.176720,0.709655,-0.330490,0.174836,0.449544,-0.180634,0.101632,0.608083,0.033174
4,0.375633,0.486405,-0.060306,0.531574,0.517183,-0.026581,0.175933,0.455786,0.144681,0.171448,0.100744,-0.045049,0.577577,-0.125687,0.147641,0.411426,-0.156426,0.289053,0.492840,0.066276
5,0.020839,-0.041567,0.011069,-0.149821,-0.026581,0.168988,0.128798,0.022314,-0.008068,0.091545,0.137395,0.162124,-0.148819,0.263449,0.027146,-0.042979,0.080385,0.230520,-0.127185,0.052554
6,0.184529,0.159213,0.011833,0.052416,0.175933,0.128798,0.264909,0.242552,-0.002170,0.188309,0.213784,0.172372,0.086755,0.199049,0.060580,0.137739,-0.045437,0.332144,0.072646,0.086559
7,0.361444,0.428541,-0.046132,0.422348,0.455786,0.022314,0.242552,0.477242,0.087714,0.206297,0.167082,0.019016,0.489898,-0.016840,0.135769,0.393852,-0.124810,0.351035,0.433982,0.082767
8,0.040363,0.146414,0.085302,0.173151,0.144681,-0.008068,-0.002170,0.087714,0.178868,0.015018,0.057374,-0.025432,0.155757,-0.102220,0.085126,0.083609,0.019853,0.041995,0.119029,0.041960
9,0.171097,0.157648,-0.009751,0.097404,0.171448,0.091545,0.188309,0.206297,0.015018,0.150282,0.141622,0.107767,0.129043,0.131756,0.087841,0.141736,-0.034963,0.261256,0.108655,0.065565
